In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

team_ids = ['8586','9825','8456','8650','10252','10261','10204','10260','8654','9937','8455','8602','9826','9879','8668','10203','8678','8346','8191','8657',
             '9768','9772','9773','10264','8613','7844','6004','9764','9765','1074320','212821','158085','7842','9774','8338','1634','4531','7841',
             '8633','7732','9906','8634','8560','8315','8370','8603','10267','8306','8371','10205','8395','8302','8385','8661','9866','9910','7878','9865',
             '8517','10283','109705','9769','10272','9770','10273','9863','8287','10277','197815','9808','8702','7877','8514','9781','10276','9862','9767','1757',
             '8636','9885','8564','8524','9875','8535','8543','9857','8686','6504','9888','9891','9804','10233','7943','9876','8600','8534','8529','6480',
             '9831','9847','9829','8639','9837','8521','9830','9851','8592','8588','10249','9941','9746','9848','8689','8550','8311','9748',
             '8178','9823','10269','9789','178475','8226','9810','8358','8721','8406','9788','8697','94937','8262','8149','9911','8722','9905',
             '8640','10229','8611','10235','6433','8614','10218','9791','6413','10228','6422','8464','10219','4116','8593','8277','6601','9908',
             '8391','8595','8415','8113','10202','8071','9907','8410','9939','8414','8231','10240',
             '7978','8635','9991','9984','9987','8342','9988','9997','9985','6351','9986','9992','1773','8203','8571','10001',
             '8637','8695','1926','10188','4685','9752','10182','95749','2166','6265','9742','1931','2088','4678','95745','8622','4081','1933','106560','9750',
             '8197','9902','8463','8466','8659','8344','8667','8472','8411','8549','10194','8655','10003','8658','8427','9817','9850','10004','8401','8669','9796','8119','10172','10163',
             '7854','8372','8558','9869','8581','9867','10281','8696','8394','8474','10268','8670','4032','7876','8288','494050','8393','161771','189624','161750','96925','8554',
             '10076','10101','10083','10082','10094','161730','10077','10084','10227','10086','161727','10080','10092','10201','213534','89396','10089','10090','202757','10096','10087','10103','213596','10078','10079','10081','10088','10098',
             '9925','8548','9800','9860','8597','8485','10251','9927','8284','8649','8282','8467'] 
    #order eng1, pt1, esp1, bra1, ita1, fra1, ger1, ned1, den1, bel1, tur1, eng2, esp2, arg1, sco1
    # i made it by hand but it can be done by websscraping but wasnt being able to do so
# Create an empty list to store all player information
all_players = []

# Iterate through each team ID
for team_id in team_ids:
    try:
        # Make the API request for each team
        url = f'https://www.fotmob.com/api/teams?id={team_id}&ccode3=PRT'
        response = requests.get(url, headers={'User-Agent': 'Chrome/115.0.0.0'})
        
        # Check if the response code is 200 (OK)
        if response.status_code == 200:
            team_info = response.json()

            # Extract the team name and league ID
            team_name = team_info['details']['name']
            league_id = team_info['overview']['table'][0]['data']['leagueId']

            # Extract player information for each role, excluding "coach"
            for role, player_list in team_info['squad']:
                if role == 'coach':
                    continue  # Skip coach information
                for player in player_list:
                    player_id = player['id']
                    player_name = player['name']
                    position = player.get('role', '')  # 'role' might not exist for some players
                    nation = player['ccode']
                    all_players.append([team_id, team_name, league_id, player_id, player_name, position, nation, role])
        else:
            print(f"Error for team ID {team_id}: Response code {response.status_code}")
    except Exception as e:
        print(f"Error for team ID {team_id}: {str(e)}")

# Create a Dataframe from all player data
df = pd.DataFrame(all_players, columns=['Team_ID', 'Team_Name', 'League_ID', 'Player_ID', 'Name', 'Position', 'Nation', 'Role'])

print(df)

     Team_ID          Team_Name  League_ID  Player_ID                Name  \
0       8586  Tottenham Hotspur         47      26295         Hugo Lloris   
1       8586  Tottenham Hotspur         47      95336      Fraser Forster   
2       8586  Tottenham Hotspur         47     538501   Guglielmo Vicario   
3       8586  Tottenham Hotspur         47     795539      Alfie Whiteman   
4       8586  Tottenham Hotspur         47     862993      Brandon Austin   
...      ...                ...        ...        ...                 ...   
8760    8467      St. Johnstone         64     314399    Christopher Kane   
8761    8467      St. Johnstone         64     711363  Diallang Jaiyesimi   
8762    8467      St. Johnstone         64     951994       Luke Jephcott   
8763    8467      St. Johnstone         64    1045828      Dara Costelloe   
8764    8467      St. Johnstone         64    1552899      Filip Franczak   

         Position Nation       Role  
0     goalkeepers    FRA    keepers  

In [2]:
base_url = 'https://www.fotmob.com/api/playerData?id='
headers = {'User-Agent': 'Chrome/115.0.0.0'}

# Create an empty list to store player statistics
player_statistics = []

# Iterate through each row in the Dataframe df
for index, row in df.iterrows():
    player_id = row['Player_ID']
    player_name = row['Name']
    league_id = row['League_ID']

    response = requests.get(f"{base_url}{player_id}", headers=headers)
    
    if response.status_code == 200:
        player_info = response.json()
        
        # Check if 'careerStatistics' key exists and is not None
        if 'careerStatistics' in player_info and player_info['careerStatistics'] is not None:
            for item in player_info['careerStatistics']:
                if item['id'] == league_id:
                    print(item['id']) 
                    print(player_name)
                    
                    # Check if 'seasons' key exists
                    if 'seasons' in item:
                        for itm in item['seasons']:
                            if itm['name'] == '2023 - 2024':
                                
                                # Check if 'stats' key exists and is not empty
                                if 'stats' in itm and len(itm['stats']) > 0:
                                    stats_arr = itm['stats'][0]['statsArr']
                                    if len(stats_arr) > 0:
                                        # Create a dictionary for the player's statistics, including the player's name
                                        player_stats = {
                                            'Player_ID': player_id,
                                            'Player_Name': player_name,
                                        }
                                        player_stats.update({item[0]: item[1]['value'] for item in stats_arr})
                                        player_statistics.append(player_stats)
                else:
                    print(f"League ID {league_id} not found for player {player_name}")
        else:
            print(f"'careerStatistics' is missing or null for player {player_name}")
    else:
        print(f"Error for player ID {player_id}: Response code {response.status_code}")

# Create a Dataframe from the player statistics
player_df = pd.DataFrame(player_statistics)

print(player_df)

47
Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
League ID 47 not found for player Hugo Lloris
47
Fraser Forster
League ID 47 not found for player Fraser Forster
League ID 47 not found for player Fraser Forster
League ID 47 not found for player Fraser Forster
League ID 47 not found for player Fraser Forster
League ID 47 not found for player Fraser Forster
League ID 47 not found for player Fraser Forster
League ID 47 not found for player Fraser Forster
47
Guglielmo Vicario
League ID 47 not found for player Guglielmo Vicario
League ID 47 not found for player Guglielmo V

In [3]:
player_df.to_excel('playerStats_FOTMOB.xlsx', index=False)